In [1]:
import pandas as pd
#Import DataSets

# Read the csv file
df = pd.read_csv('data_train.csv')
y_test = pd.read_csv('test_competition.csv')


Y_train = df["default.payment.next.month"]

X_train = df.drop(columns = ["default.payment.next.month"])






In [2]:
# import joblib 

# #Splitting DataFrame
# X_train = df.iloc[:20000,:]

# Y_train = X_train["default.payment.next.month"]

# X_train = X_train.drop(columns = ["default.payment.next.month"])



# y_test = df.iloc[20000:,:]

# y_test.to_csv("test_5000.csv", index=False)

# y_test = y_test.drop(columns = ["default.payment.next.month"])
# print(y_test)



# # # 
# # y_test = df.iloc[20000:,:]
# # y_test = y_test.drop(columns = ["default.payment.next.month"])



In [3]:
# #Using smote to overSample minorities in the dataset for avoiding bias
# from imblearn.over_sampling import SMOTE

# print("Smoting...")

# # Step 2: Apply SMOTE to the training data
# smote = SMOTE(random_state=42)
# X_train, Y_train = smote.fit_resample(X_train, Y_train)

# print("Data over sampled !")



In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

def bestModelFindingWithRandomForest(param_grid, scoring, X_train, Y_train, X_train_reshape=False, Y_train_reshape=False):
    """
    Find best Random Forest model using GridSearchCV
    
    Parameters:
    -----------
    param_grid : dict
        Parameter grid for GridSearchCV
    scoring : str
        Scoring metric ('r2', 'neg_mean_squared_error', etc.)
    X_train : array-like
        Training features
    Y_train : array-like
        Training labels
    X_train_reshape : bool
        Whether to reshape X_train
    Y_train_reshape : bool
        Whether to reshape Y_train
    """
    try:
        print("Loading RandomForestRegressor...")
        model = RandomForestRegressor(random_state=100)
        
        if X_train_reshape:
            print("Reshaping X_train...")
            X_train = X_train.reshape(X_train.shape[0], -1)
            
        if Y_train_reshape:
            print("Reshaping Y_train...")
            Y_train = Y_train.reshape(Y_train.shape[0], -1)
            
        print("Loading GridSearchCV model...")
        # Using GridSearchCV with 10-fold cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=10,
            scoring=scoring,
            n_jobs=-1,
            verbose=1
        )
        
        print("Fitting GridSearchCV model...")
        # Fit the model to find the best hyperparameters
        grid_search.fit(X_train, Y_train)
        
        print("GridSearchCV fitted")
        
        # Best parameters and score obtained
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_
        
        # Format score based on metric
        if scoring.startswith('neg_'):
            best_score = -best_score  # Convert negative metrics back to positive
        
        print("\nBest parameters found:")
        for param, value in best_params.items():
            print(f"{param}: {value}")
        print(f"\nBest {scoring} score: {best_score:.4f}")
        
        return grid_search.best_estimator_
        
    except Exception as e:
        print(f"{str(e)}")
        return None

# Define the parameter grid for regression
param_grid = {
    'n_estimators': [100, 150, 200],           # Number of trees
    'max_depth': [10, 15, 20, 25],             # Tree depth
    'min_samples_split': [2, 5, 10],           # Minimum samples required to split
    'min_samples_leaf': [1, 2, 4],             # Minimum samples required in leaf
    'max_features': ['auto', 'sqrt']           # Number of features to consider
}

bestModel = bestModelFindingWithRandomForest(
    param_grid=param_grid,
    scoring='r2',  # or 'neg_mean_squared_error', 'neg_mean_absolute_error'
    X_train=X_train,
    Y_train=Y_train
)


Loading RandomForestRegressor...
Loading GridSearchCV model...
Fitting GridSearchCV model...
Fitting 10 folds for each of 216 candidates, totalling 2160 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
1080 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
427 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/anaconda3/lib/python3.12/site

GridSearchCV fitted

Best parameters found:
max_depth: 10
max_features: sqrt
min_samples_leaf: 1
min_samples_split: 2
n_estimators: 200

Best r2 score: 0.2229


In [5]:
!pip install lightgbm

In [16]:
# METHOD 2
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV

def bestModelFinding(param_grid, scoring, X_train, Y_train):
    """
    Find the best LightGBM model using GridSearchCV for regression tasks
    
    Parameters:
    -----------
    param_grid : dict
        Parameter grid for GridSearchCV
    scoring : str
        Scoring metric ('neg_mean_squared_error', 'r2', etc.)
    X_train : array-like
        Training features
    Y_train : array-like
        Training labels
    """
    # Base parameters for regression
    base_params = {
        'objective': 'regression',
        'metric': 'rmse',  # Root Mean Square Error
        'verbosity': -1,
        'random_state': 42
    }
    
    try:
        # Loading model
        print("Loading lightgbm...")
        # Initialize the LGBMRegressor with base parameters
        model = lgb.LGBMRegressor(**base_params)
        
        print("Loading GridSearch model...")
        # Configure GridSearchCV with 5-fold cross-validation
        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=5,
            scoring=scoring,
            n_jobs=-1,
            verbose=1  # for progress tracking
        )
        
        print("Fitting GridSearch model...")
        # Fit the GridSearch model to find the best hyperparameters
        grid_search.fit(X_train, Y_train)
        print("GridSearch fitted")
        
        # Best parameters and best score found
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_ * 100  # In percentage 

        print("\nBest parameters found:")
        for param, value in best_params.items():
            print(f"{param}: {value}")
        print(f"\nBest {scoring} score: {best_score:.2f}%")
        
        return grid_search.best_estimator_
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

# Example usage:

# Define your parameter grid
param_grid = {
    'n_estimators': [200,600],
    'learning_rate': [0.01, 0.1 , 0.08],
    'max_depth': [2,5, 10, 25,30,90, 100],
    'num_leaves': [30,63, 100, 120]
}

# Find the best model
bestModel = bestModelFinding(param_grid, "neg_mean_squared_error", X_train, Y_train)


Loading lightgbm...
Loading GridSearch model...
Fitting GridSearch model...
Fitting 5 folds for each of 168 candidates, totalling 840 fits


KeyboardInterrupt: 

In [17]:
import pandas as pd
predicted_payment_value = bestModel.predict(y_test)
print(predicted_payment_value)

# Assuming y_test is a DataFrame containing the 'ID' column
predicted_df = pd.DataFrame({
    'ID': y_test['ID'],       # Retrieve the 'ID' column from y_test
    'default.probability': predicted_payment_value  # Predicted payment values from the model
})

# Sort predicted_df by the 'PAYED' column in descending order and extract sorted IDs
sorted_df = predicted_df.sort_values(by='default.probability', ascending=False)

# Display the resulting DataFrame of sorted IDs and save as a CSV
sorted_df.head(1000).to_csv("sorted_df.csv", index=False)
print("Saved sorted predictions to sorted_df.csv")


[0.0653004  0.08790907 0.5755109  ... 0.37429492 0.76042562 0.25904249]
Saved sorted predictions to sorted_df.csv


In [ ]:
import pandas as pd 

# Prediction step
print("Predicting...")
predicted_survival = bestModel.predict_proba(y_test)  

# Extracting the first column of predicted probabilities
unsortedPredictedPaymentValue = predicted_survival[:, 1]

# Combining Passenger ID and predicted payment probabilities into a DataFrame
predicted_df = pd.DataFrame({
    'ID': y_test['ID'],      # Get the PassengerId from y_test
    'PAYED': unsortedPredictedPaymentValue  # Add the predicted payment probability (class 0)
})

# Sort predicted_df by the 'PAYED' column in descending order and extract sorted IDs
sorted_df = predicted_df.sort_values(by='PAYED', ascending=False)[['ID']]

# Display the resulting DataFrame of sorted IDs

sorted_df.head(1000).to_csv("sorted_df.csv", index=False)



Predicting...


AttributeError: 'LGBMRegressor' object has no attribute 'predict_proba'